<a href="https://colab.research.google.com/github/KhanilReddy/ML1/blob/main/Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Downloading the entire project folder "Qualcomm-DL-Hackathon" from GitHub to your computer/colab
!git clone https://github.com/Prashant-AV/Qualcomm-DL-Hackathon.git

Cloning into 'Qualcomm-DL-Hackathon'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (10/10), 30.68 MiB | 30.74 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [2]:
import zipfile
import os
import shutil

def extract_all_files(zip_dir, output_dir):
    """
        This function is to extract all files from ZIP archives located in a specified directory and place them into an output directory.

        Args:
            zip_dir (:class:`str`): : Required: Directory path in which all zip files are present
            output_dir (:class:`str`): Required: Directory path in which all images has to be stored

        Example : Extracted all the images from images part-1.zip and images part-2.zip to all_images folder.
    """
    if os.path.exists(zip_dir):
      # Get the zip files in zip_dir directory
      files = os.listdir(zip_dir)
      zip_files = [f for f in files if f.endswith('.zip')]

      if not os.path.exists(output_dir):
        os.mkdir(output_dir)

      # Extract each zip file
      for zip_file in zip_files:
          with zipfile.ZipFile(os.path.join(zip_dir, zip_file), 'r') as zip_ref:
              for member in zip_ref.namelist():
                  # Extract each file to the output_dir directory without creating subfolders
                  filename = os.path.basename(member)
                  if filename:
                      source = zip_ref.open(member)
                      target = open(os.path.join(output_dir, filename), "wb")
                      with source, target:
                          shutil.copyfileobj(source, target)
    else:
      print("f{zip_dir} doesn't exits")

In [3]:
# Define the directory containing the zip files and the all_images directory
train_directory = '/content/Qualcomm-DL-Hackathon/train/'
all_images_directory = os.path.join(train_directory , 'all_images')

# Extract all files from the zip files in path /content/Qualcomm-DL-Hackathon/train/ to the /content/Qualcomm-DL-Hackathon/train/all_images directory
extract_all_files(train_directory, all_images_directory)

print(f"All files have been extracted to {all_images_directory}.")

All files have been extracted to /content/Qualcomm-DL-Hackathon/train/all_images.


In [19]:
# pandas to read the csv and write the predicted data to csv
import pandas as pd

# Importing necessary modules from TensorFlow and Keras
from tensorflow import keras
from keras.layers import Dense, Dropout, Flatten
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.utils import to_categorical
from keras.models import Sequential
from keras.optimizers import SGD
from keras.models import Model

# Importing necessary modules from scikit-learn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Importing tqdm for progress bar and numpy for numerical operations
from tqdm import tqdm
import numpy as np



In [20]:
# Loading the training data from a CSV file
train = pd.read_csv('/content/Qualcomm-DL-Hackathon/train/train.csv')

# Initializing an empty list to store the training images
train_img=[]

for i in tqdm(range(train.shape[0])):
  # Loading the image and resizing it to 224x224 pixels, Converting the image to an array and Appending the image array to the list
  temp_img = image.load_img('/content/Qualcomm-DL-Hackathon/train/all_images/'+train['image_names'][i], target_size=(224,224))
  temp_img=image.img_to_array(temp_img)

  train_img.append(temp_img)


100%|██████████| 1646/1646 [00:01<00:00, 1006.31it/s]


In [21]:

def vgg16_model(img_rows, img_cols, channel=1, num_classes=None):
    """
    Builds a VGG16-based convolutional neural network model with custom top layers.

    Parameters:
    img_rows (int): The number of rows in the input images.
    img_cols (int): The number of columns in the input images.
    channel (int): The number of channels in the input images (default is 1).
    num_classes (int): The number of output classes for the classification task.

    Returns:
    keras.models.Model: A compiled Keras model ready for training.


    Example usage:
    model = vgg16_model(img_rows=224, img_cols=224, channel=3, num_classes=10)
    """
    #Loads the VGG16 model pre-trained on ImageNet, excluding the top fully connected layers
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_rows, img_cols, channel))
    base_model.summary()

    # Freeze the layers of the base model to prevent them from being trained
    for layer in base_model.layers:
        layer.trainable = False

    # Added custom layers on top of the base model
    x = base_model.output
    x = Flatten()(x)                                                            # Flatten the output of the base model
    x = Dense(512, activation='relu')(x)                                        # Added a fully connected layer with 512 units and ReLU activation
    x = Dropout(0.5)(x)                                                         # Added a dropout layer with a 50% dropout rate to prevent overfitting
    x = Dense(num_classes, activation='softmax')(x)                             # Added the output layer with softmax activation for classification

    # Create the new model
    model = Model(inputs=base_model.input, outputs=x)

    # Compile the model with SGD optimizer and binary cross-entropy loss
    sgd = SGD(learning_rate=1e-3, weight_decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

    model.summary()

    return model

In [22]:
# Extracting the target variable from the training data
train_y = train['emergency_or_not'].values

# Initializing the LabelEncoder
le = LabelEncoder()

# Encoding the target variable
train_y = le.fit_transform(train_y)

# Converting the encoded labels to categorical format
train_y=to_categorical(train_y)
train_y=np.array(train_y)

# Splitting the data into training and validation sets
X_train, X_valid, Y_train, Y_valid=train_test_split(train_img,train_y,test_size=0.2, random_state=42)

# Setting parameters for the model
img_rows, img_cols = 224, 224           # Resolution of inputs
channel = 3                             # Number of channels (3 for RGB images)
num_classes = 2                         # Number of output classes : emergency and Non_emergency
batch_size = 32                         # Batch size for training
nb_epoch = 15                           # Number of epochs for training

# Converting lists to numpy arrays so that they can be efficiently processed by the deep learning model during training and validation.
X_train = np.stack(X_train)
X_valid = np.stack(X_valid)
Y_train = np.stack(Y_train)
Y_valid = np.stack(Y_valid)

# Load our final model
model = vgg16_model(img_rows, img_cols, channel, num_classes)

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 512)                 │      12,845,568 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 27,561,282 (105.14 MB)

 Trainable params: 12,846,594 (49.01 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [23]:
# Train the model with necessary input parameters such as input images, labels, batch_size, epochs,
model.fit(X_train, Y_train,batch_size=batch_size,epochs=nb_epoch,shuffle=True,verbose=1,validation_data=(X_valid, Y_valid))

# Make predictions on the validation set
predictions_valid = model.predict(X_valid, batch_size=batch_size, verbose=1)

# Get the class with the highest probability for each prediction
validation_prediction = np.argmax(predictions_valid, axis=1)
validation_prediction

Epoch 1/15
42/42 ━━━━━━━━━━━━━━━━━━━━ 11s 212ms/step - accuracy: 0.7468 - loss: 15.3391 - val_accuracy: 0.8303 - val_loss: 0.3864
Epoch 2/15
42/42 ━━━━━━━━━━━━━━━━━━━━ 7s 159ms/step - accuracy: 0.9077 - loss: 0.3368 - val_accuracy: 0.9242 - val_loss: 0.1758
Epoch 3/15
42/42 ━━━━━━━━━━━━━━━━━━━━ 8s 190ms/step - accuracy: 0.9536 - loss: 0.1526 - val_accuracy: 0.9030 - val_loss: 0.2785
Epoch 4/15
42/42 ━━━━━━━━━━━━━━━━━━━━ 10s 192ms/step - accuracy: 0.9610 - loss: 0.1064 - val_accuracy: 0.9242 - val_loss: 0.1863
Epoch 5/15
42/42 ━━━━━━━━━━━━━━━━━━━━ 9s 169ms/step - accuracy: 0.9805 - loss: 0.0796 - val_accuracy: 0.9273 - val_loss: 0.2204
Epoch 6/15
42/42 ━━━━━━━━━━━━━━━━━━━━ 10s 168ms/step - accuracy: 0.9619 - loss: 0.0803 - val_accuracy: 0.9273 - val_loss: 0.1754
Epoch 7/15
42/42 ━━━━━━━━━━━━━━━━━━━━ 10s 167ms/step - accuracy: 0.9713 - loss: 0.0652 - val_accuracy: 0.9242 - val_loss: 0.2168
Epoch 8/15
42/42 ━━━━━━━━━━━━━━━━━━━━ 10s 165ms/step - accuracy: 0.9813 - loss: 0.0510 - val_accura

array([1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,

In [24]:
# Load the test data from a CSV file
test = pd.read_csv('/content/Qualcomm-DL-Hackathon/test.csv')

# Initialize an empty list to store the test images
test_image = []
for i in tqdm(range(test.shape[0])):
    # Loading the image and resizing it to 224x224 pixels, Converting the image to an array and Appending the image array to the list
    img = image.load_img('/content/Qualcomm-DL-Hackathon/train/all_images/'+test['image_names'][i], target_size=(224,224))
    img = image.img_to_array(img)
    test_image.append(img)

# Convert the list of test images to a numpy array
test_image = np.stack(test_image)
test_image.shape

# Make predictions on the test images
predictions_test = model.predict(test_image, batch_size=batch_size, verbose=1)
predictions_test

# Get the class with the highest probability for each prediction
test_prediction = np.argmax(predictions_test, axis=1)
test_prediction

100%|██████████| 706/706 [00:00<00:00, 1418.28it/s]


23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 134ms/step


array([0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0,
       1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1,

In [25]:
# Add the predictions to the test data
test['emergency_or_not'] = test_prediction

# Save the updated test data with predictions to a new CSV file
test.to_csv('/content/Qualcomm-DL-Hackathon/submission_sgd_15_32_512_drop_softmax.csv', header=True, index=False)